In [98]:
!pip install gspread
!pip install gspread-dataframe


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [127]:
import requests
from bs4 import BeautifulSoup
import gspread 
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
import json

スプレッドシートと連携

In [100]:

SP_CREDENTIAL_FILE="C:\\Users\kouhei_shimakura\\楽天トラベルAPI\\Scraping\\gspread-test-399023-650ba3322327.json"
SP_COPE=[
  'https://www.googleapis.com/auth/drive',
  'https://spreadsheets.google.com/feeds'
]
SP_SHEET_KEY='1BzQ19jeu8m6hpW9MYXRCDM9Q8S5VhKWTQV-pL6a9ngI'
SP_SHEET='shinjuku1'

In [101]:
credentials = Credentials.from_service_account_file(SP_CREDENTIAL_FILE, scopes=SP_COPE)
gc = gspread.authorize(credentials)

HTML取得

In [102]:
url = 'http://suumo.jp/chintai/tokyo/sc_shinjuku/?page={}'
target_url = url.format(1)
print(target_url)

http://suumo.jp/chintai/tokyo/sc_shinjuku/?page=1


In [103]:
r = requests.get(target_url)
soup = BeautifulSoup(r.text)

In [104]:
r

<Response [200]>

1ページあたりの物件情報を取得

In [105]:
contents = soup.find_all('div',class_='cassetteitem')

In [106]:
len(contents)

20

In [107]:
content = contents[0]
content

<div class="cassetteitem">
<div class="cassetteitem-detail">
<div class="cassetteitem-detail-object">
<div class="cassetteitem_object">
<div class="cassetteitem_object-item">
<img alt="" class="js-noContextMenu js-linkImage js-scrollLazy js-adjustImg" height="0" rel="https://img01.suumo.com/front/gazo/fr/bukken/590/100344780590/100344780590_gw.jpg" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAAAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw%3D%3D" width="0"/>
</div>
</div>
</div>
<div class="cassetteitem-detail-body">
<div class="cassetteitem_content">
<div class="cassetteitem_content-label"><span class="ui-pct ui-pct--util1">賃貸マンション</span></div>
<div class="cassetteitem_content-title">フラッツ神楽坂</div>
<div class="cassetteitem_content-body">
<ul class="cassetteitem_detail">
<li class="cassetteitem_detail-col1">東京都新宿区矢来町</li>
<li class="cassetteitem_detail-col2">
<div class="cassetteitem_detail-text">東京メトロ東西線/神楽坂駅 歩2分</div>
<div class="cassetteitem_detail-text">都営大江戸線/牛込神楽坂駅 歩8分</div>
<div class="

物件情報
・物件名
・住所
・アドレス
・築年数
部屋番号
・物件の階数
・物件の賃料/管理費
・物件の敷金・礼金
・物件の間取り・面積

In [108]:
#物件情報・建物情報を変数detailに格納する
detail = content.find('div',class_ ='cassetteitem-detail')

#各部屋の情報を変数tableに格納する
table = content.find('table',class_ ='cassetteitem_other')

In [109]:
#変数titleに、物件名を格納する
title = detail.find('div',class_='cassetteitem_content-title').text

#変数adressに住所を格納する
address = detail.find('li',class_='cassetteitem_detail-col1').text

#変数accessにアクセス情報を格納する
access = detail.find('li',class_='cassetteitem_detail-col2').text

#変数ageに築年数を格納する
age = detail.find('li',class_='cassetteitem_detail-col3').text


In [110]:
#各変数の取得結果を確認
title,address,access,age

('フラッツ神楽坂',
 '東京都新宿区矢来町',
 '\n東京メトロ東西線/神楽坂駅 歩2分\n都営大江戸線/牛込神楽坂駅 歩8分\nＪＲ中央線/飯田橋駅 歩15分\n',
 '\n築10年\n3階建\n')

In [111]:
table

<table class="cassetteitem_other">
<thead>
<tr>
<th class="cassetteitem_other-col01"> </th>
<th class="cassetteitem_other-col02"> </th>
<th class="cassetteitem_other-col03">階</th>
<th class="cassetteitem_other-col04">賃料/管理費</th>
<th class="cassetteitem_other-col05">敷金/礼金</th>
<th class="cassetteitem_other-col06">間取り/専有面積</th>
<th class="cassetteitem_other-col07"> </th>
<th class="cassetteitem_other-col08">お気に入り</th>
<th class="cassetteitem_other-col09"> </th>
</tr>
</thead>
<tbody>
<tr class="js-cassette_link">
<td class="cassetteitem_other-checkbox cassetteitem_other-checkbox--newarrival js-cassetteitem_checkbox">
<input class="js-ikkatsuCB js-single_checkbox" id="bukken_0" name="bc" type="checkbox" value="100344780590"/><label for="bc"> </label>
</td>
<td>
<div class="casssetteitem_other-thumbnail js-view_gallery_images js-noContextMenu" data-imgs="https://img01.suumo.com/front/gazo/fr/bukken/590/100344780590/100344780590_go.jpg,https://img01.suumo.com/front/gazo/fr/bukken/590/100344

部屋情報を抽出する
・物件の階数
・物件の賃料/管理費
・物件の敷金・礼金
・物件の間取り・面積

各部屋の情報は<table>タグに囲まれているので<tr>を見てあげると１つの部屋情報になっている

In [112]:
#変数tableからすべてのtrタグを取得して、変数tr_tagsに格納
tr_tags = table.find_all('tr',class_='js-cassette_link')

#tr_tagsの中から最初の１つだけtr_tagに格納
tr_tag = tr_tags[0]

変数tr_tagから、以下の情報を取得する
・物件の階数
・物件の賃料管理費
・物件の敷金・礼金
・物件の間取り・面積

In [113]:
#変数floor,price,first_fee.capacityに４つの情報を格納する
#unpack
floor,price,first_fee,capacity = tr_tag.find_all('td')[2:6]

In [114]:
#floor,price,first_fee,capacityの中身を確認する
floor,price,first_fee,capacity

(<td>
 											1階</td>,
 <td>
 <ul>
 <li><span class="cassetteitem_price cassetteitem_price--rent"><span class="cassetteitem_other-emphasis ui-text--bold">16万円</span></span></li>
 <li><span class="cassetteitem_price cassetteitem_price--administration">2000円</span></li>
 </ul>
 </td>,
 <td>
 <ul>
 <li><span class="cassetteitem_price cassetteitem_price--deposit">16万円</span></li>
 <li><span class="cassetteitem_price cassetteitem_price--gratuity">16万円</span></li>
 </ul>
 </td>,
 <td>
 <ul>
 <li><span class="cassetteitem_madori">2SLDK</span></li>
 <li><span class="cassetteitem_menseki">61.73m<sup>2</sup></span></li>
 </ul>
 </td>)

In [115]:
#unpack
#変数feeとmanagement_feeに、賃料と管理費を格納する
fee,management_fee = price.find_all('li')

#変数depositとgratuity
deposit,gratuity = first_fee.find_all('li')
                                      
#変数madoriとmensekiに、間取りと面積を格納する
madori,menseki = capacity.find_all('li')                                      


In [116]:
print(fee)
print(management_fee)
print()

print(deposit)
print(gratuity)
print()

print(madori)
print(menseki)

<li><span class="cassetteitem_price cassetteitem_price--rent"><span class="cassetteitem_other-emphasis ui-text--bold">16万円</span></span></li>
<li><span class="cassetteitem_price cassetteitem_price--administration">2000円</span></li>

<li><span class="cassetteitem_price cassetteitem_price--deposit">16万円</span></li>
<li><span class="cassetteitem_price cassetteitem_price--gratuity">16万円</span></li>

<li><span class="cassetteitem_madori">2SLDK</span></li>
<li><span class="cassetteitem_menseki">61.73m<sup>2</sup></span></li>


取得した情報を辞書に格納する
・title
・address
・access
・age
・floor
・fee
・management_fee
・deposit
・gratuity
・madori
・menseki

In [117]:
d={
 'title':title,
 'address':address,
 'access':access,
 'age':age,
 'floor':floor.text,
 'fee':fee.text,
 'management_fee':management_fee.text,
 'deposit':deposit.text,
 'gratuity':gratuity.text,
 'madori':madori.text,
 'menseki':menseki.text
}    

In [118]:
d

{'title': 'フラッツ神楽坂',
 'address': '東京都新宿区矢来町',
 'access': '\n東京メトロ東西線/神楽坂駅 歩2分\n都営大江戸線/牛込神楽坂駅 歩8分\nＪＲ中央線/飯田橋駅 歩15分\n',
 'age': '\n築10年\n3階建\n',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t1階',
 'fee': '16万円',
 'management_fee': '2000円',
 'deposit': '16万円',
 'gratuity': '16万円',
 'madori': '2SLDK',
 'menseki': '61.73m2'}

1ページ目からすべての情報を取得する
forループをつかっていくので、それぞれの部屋情報を格納できるように、変数d_listという名前でリストを準備しておく

In [119]:
#変数d_listに空のリストを作成する
d_list = []

#すべての物件情報（20件）を取得する
contents = soup.find_all('div',class_='cassetteitem')

#各物件情報をforループで取得する
for content in contents:
    
    #物件情報と部屋情報を取得しておく
    detail = content.find('div',class_ ='cassetteitem-detail')
    table = content.find('table',class_ ='cassetteitem_other')
    
    #物件情報から必要な情報を取得する
    title = detail.find('div',class_='cassetteitem_content-title').text
    address = detail.find('li',class_='cassetteitem_detail-col1').text
    access = detail.find('li',class_='cassetteitem_detail-col2').text
    age = detail.find('li',class_='cassetteitem_detail-col3').text
    
    #部屋情報のブロックから、各部屋情報を取得する
    tr_tags = table.find_all('tr',class_='js-cassette_link')
    
    #部屋情報をforループで取得する
    for tr_tag in tr_tags:
        
        #部屋情報の行から、欲しい情報を取得する
        floor,price,first_fee,capacity = tr_tag.find_all('td')[2:6]
        
        #さらに細かい情報を取得する
        fee,management_fee = price.find_all('li')
        deposit,gratuity = first_fee.find_all('li')
        madori,menseki = capacity.find_all('li')                                      
        
        #取得したすべての情報を辞書に格納する
        d={
         'title':title,
         'address':address,
         'access':access,
         'age':age,
         'floor':floor.text,
         'fee':fee.text,
         'management_fee':management_fee.text,
         'deposit':deposit.text,
         'gratuity':gratuity.text,
         'madori':madori.text,
         'menseki':menseki.text
        }    
        
        #取得した辞書をd_listに格納する
        d_list.append(d)
        

pandasでデータフレーム化を実施

In [120]:
import pandas as pd

In [121]:
df = pd.DataFrame(d_list)

In [122]:
df

,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,フラッツ神楽坂,東京都新宿区矢来町,\n東京メトロ東西線/神楽坂駅 歩2分\n都営大江戸線/牛込神楽坂駅 歩8分\nＪＲ中央線/...,\n築10年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,16万円,2000円,16万円,16万円,2SLDK,61.73m2
1,THE CENTER TOKYO,東京都新宿区市谷本村町,\n都営新宿線/曙橋駅 歩5分\nＪＲ中央線/市ケ谷駅 歩15分\n都営大江戸線/牛込柳町駅...,\n築16年\n地下2地上38階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t16階,39万円,20000円,39万円,39万円,3LDK,84.77m2
2,THE CENTER TOKYO,東京都新宿区市谷本村町,\n都営新宿線/曙橋駅 歩5分\nＪＲ中央線/市ケ谷駅 歩15分\n都営大江戸線/牛込柳町駅...,\n築16年\n地下2地上38階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t29階,42万円,-,84万円,42万円,1SLDK,87.14m2
3,ザ・パークハウス新宿柏木,東京都新宿区北新宿３,\nＪＲ中央線/大久保駅 歩3分\nＪＲ山手線/新大久保駅 歩7分\n西武新宿線/西武新宿駅...,\n築12年\n13階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,23万円,10000円,23万円,23万円,2LDK,52.52m2
4,ザ・パークハウス新宿柏木,東京都新宿区北新宿３,\nＪＲ中央線/大久保駅 歩3分\nＪＲ山手線/新大久保駅 歩7分\n西武新宿線/西武新宿駅...,\n築12年\n13階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,29.5万円,15000円,29.5万円,29.5万円,3LDK,71.31m2
...,...,...,...,...,...,...,...,...,...,...,...
170,都営大江戸線 西新宿五丁目駅 地下2地上60階建 築7年,東京都新宿区西新宿５,\n都営大江戸線/西新宿五丁目駅 歩7分\n東京メトロ丸ノ内線/西新宿駅 歩9分\n都営大江...,\n築7年\n地下2地上60階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t56階,48.5万円,10000円,97万円,48.5万円,2LDK,89.31m2
171,都営大江戸線 西新宿五丁目駅 地下2地上60階建 築7年,東京都新宿区西新宿５,\n都営大江戸線/西新宿五丁目駅 歩7分\n東京メトロ丸ノ内線/西新宿駅 歩9分\n都営大江...,\n築7年\n地下2地上60階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t56階,48.5万円,10000円,97万円,48.5万円,2LDK,89.31m2
172,都営大江戸線 西新宿五丁目駅 地下2地上60階建 築7年,東京都新宿区西新宿５,\n都営大江戸線/西新宿五丁目駅 歩7分\n東京メトロ丸ノ内線/西新宿駅 歩9分\n都営大江...,\n築7年\n地下2地上60階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t52階,48万円,20000円,48万円,48万円,3SLDK,89.86m2
173,パークハウス牛込北町,東京都新宿区北町,\n都営大江戸線/牛込神楽坂駅 歩4分\n東京メトロ東西線/神楽坂駅 歩10分\nＪＲ総武線...,\n築13年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,20.9万円,10000円,20.9万円,20.9万円,1LDK,43m2


In [123]:
pwd

'C:\\Users\\kouhei_shimakura\\楽天トラベルAPI\\Scraping'

In [124]:
df.shape

(175, 11)

In [125]:
#df.to_csv('shinjuku1.csv',index=None,encoding='utf-8-sig')

In [126]:
import re

def extract_number(s):
    if not isinstance(s, str):  # 文字列でない場合はそのまま返す
        return s

    match = re.search(r'([\d.]+)', s)

    if match is None:
        return None

    value = match.group(1)

    if '万' in s:
        return float(value) * 10000

    return float(value)



columns_to_modify = ['fee', 'management_fee', 'deposit', 'gratuity', 'menseki']

for col in columns_to_modify:
    df[col] = df[col].apply(extract_number)

# madoriは少し異なるので別途処理
def extract_madori(s):
    rooms = re.findall(r'(\d)[SLDK]', s)
    return int(rooms[0]) if rooms else None

df['madori'] = df['madori'].apply(extract_madori)


In [133]:
df

,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,フラッツ神楽坂,東京都新宿区矢来町,\n東京メトロ東西線/神楽坂駅 歩2分\n都営大江戸線/牛込神楽坂駅 歩8分\nＪＲ中央線/...,\n築10年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,160000.0,2000.0,160000.0,160000.0,2.0,61.73
1,THE CENTER TOKYO,東京都新宿区市谷本村町,\n都営新宿線/曙橋駅 歩5分\nＪＲ中央線/市ケ谷駅 歩15分\n都営大江戸線/牛込柳町駅...,\n築16年\n地下2地上38階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t16階,390000.0,20000.0,390000.0,390000.0,3.0,84.77
2,THE CENTER TOKYO,東京都新宿区市谷本村町,\n都営新宿線/曙橋駅 歩5分\nＪＲ中央線/市ケ谷駅 歩15分\n都営大江戸線/牛込柳町駅...,\n築16年\n地下2地上38階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t29階,420000.0,NaN,840000.0,420000.0,1.0,87.14
3,ザ・パークハウス新宿柏木,東京都新宿区北新宿３,\nＪＲ中央線/大久保駅 歩3分\nＪＲ山手線/新大久保駅 歩7分\n西武新宿線/西武新宿駅...,\n築12年\n13階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,230000.0,10000.0,230000.0,230000.0,2.0,52.52
4,ザ・パークハウス新宿柏木,東京都新宿区北新宿３,\nＪＲ中央線/大久保駅 歩3分\nＪＲ山手線/新大久保駅 歩7分\n西武新宿線/西武新宿駅...,\n築12年\n13階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,295000.0,15000.0,295000.0,295000.0,3.0,71.31
...,...,...,...,...,...,...,...,...,...,...,...
170,都営大江戸線 西新宿五丁目駅 地下2地上60階建 築7年,東京都新宿区西新宿５,\n都営大江戸線/西新宿五丁目駅 歩7分\n東京メトロ丸ノ内線/西新宿駅 歩9分\n都営大江...,\n築7年\n地下2地上60階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t56階,485000.0,10000.0,970000.0,485000.0,2.0,89.31
171,都営大江戸線 西新宿五丁目駅 地下2地上60階建 築7年,東京都新宿区西新宿５,\n都営大江戸線/西新宿五丁目駅 歩7分\n東京メトロ丸ノ内線/西新宿駅 歩9分\n都営大江...,\n築7年\n地下2地上60階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t56階,485000.0,10000.0,970000.0,485000.0,2.0,89.31
172,都営大江戸線 西新宿五丁目駅 地下2地上60階建 築7年,東京都新宿区西新宿５,\n都営大江戸線/西新宿五丁目駅 歩7分\n東京メトロ丸ノ内線/西新宿駅 歩9分\n都営大江...,\n築7年\n地下2地上60階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t52階,480000.0,20000.0,480000.0,480000.0,3.0,89.86
173,パークハウス牛込北町,東京都新宿区北町,\n都営大江戸線/牛込神楽坂駅 歩4分\n東京メトロ東西線/神楽坂駅 歩10分\nＪＲ総武線...,\n築13年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,209000.0,10000.0,209000.0,209000.0,1.0,43.00


In [134]:
sh = gc.open_by_key(SP_SHEET_KEY)
worksheet = sh.worksheet(SP_SHEET)
worksheet.clear() 

{'spreadsheetId': '1BzQ19jeu8m6hpW9MYXRCDM9Q8S5VhKWTQV-pL6a9ngI',
 'clearedRange': 'shinjuku1!A1:Z1000'}

In [140]:
df = df.fillna(0)

In [141]:
# DataFrameをリストのリストに変換
values = [df.columns.values.tolist()] + df.values.tolist()

# スプレッドシートを更新
worksheet.update('A1', values)


{'spreadsheetId': '1BzQ19jeu8m6hpW9MYXRCDM9Q8S5VhKWTQV-pL6a9ngI',
 'updatedRange': 'shinjuku1!A1:K176',
 'updatedRows': 176,
 'updatedColumns': 11,
 'updatedCells': 1936}